- On separate 10000 testing set, the retrained model has average accuracy of 84.5% and number of discrepancies is 180. However, without any retraining with adversarial inputs, the accuracy is 81.8% on average and number of discrepancies is 311. 

In [68]:
import numpy as np
from mnist import MNIST
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from PIL import Image
from matplotlib import pyplot as plt

In [69]:
def shuffle(X, y):
    permutation = np.arange(X.shape[0])
    np.random.shuffle(permutation)
    return X[permutation], y[permutation]

def load_dataset():
    mndata = MNIST('./data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    X_test, labels_test = map(np.array, mndata.load_testing())
    X_train = X_train/255.
    X_test = X_test/255.
    return X_train, labels_train, X_test, labels_test

def get_scene(img, proj):
    return np.dot(img, proj.T)

# Transform the image vectors into the hypervectors
def get_scenes(images, proj):
    print(images.shape)
    print(proj.shape)
    return np.dot(images, proj.T)

def classify(images, digit_vectors):
    similarities = cosine_similarity(images, digit_vectors)
    classifications = np.argmax(similarities, axis=1)
    return classifications

In [70]:
X_train, labels_train, X_valid, labels_valid = load_dataset()
# X_train, labels_train = shuffle(X_train, labels_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, labels_train, test_size=0.33, random_state=42)

In [71]:
def HD_classifiers(seed, encoding="float"):
    # print("Generating random projection...")
    # proj = np.random.rand(D, IMG_LEN * IMG_LEN)
    print("Seed: ", seed)
    print("Encoding: ", encoding)
    print("Generating random projection...")
    np.random.seed(seed)
    proj = np.random.rand(D, IMG_LEN * IMG_LEN)
    if encoding == "bipolar":
        proj[proj==0] = -1
    print(proj.shape)
    print("Projecting images to higher dim space...")
    X_train_copy = get_scenes(X_train, proj)
    
    digit_vectors = np.zeros((10, D))
    print("Dimension of digit vector: ", digit_vectors.shape)
    
    for i in range(NUM_SAMPLES):
        digit_vectors[y_train[i]] += X_train_copy[i]
    digit_vectors = np.array(digit_vectors)
    
    if encoding == "bipolar":
        digit_vectors[digit_vectors > 0] = 1
        digit_vectors[digit_vectors <= 0] = -1
    
    predictions = classify(X_train_copy, digit_vectors)
    acc = accuracy_score(y_train[:X_train_copy.shape[0]], predictions)
    print("Train accuracy: ", acc)
    
    X_test_copy = get_scenes(X_test, proj)
    predictions = classify(X_test_copy, digit_vectors)
    acc = accuracy_score(y_test[:X_test_copy.shape[0]], predictions)
    print("Test accuracy: ", acc)
        
    return digit_vectors, proj, X_train_copy, X_test_copy

In [72]:
D = 10000 # dimensions in random space
IMG_LEN = 28
NUM_SAMPLES = X_train.shape[0]
seeds = [30, 40, 50]
epochs = 20

models = []
projs = []
X_train_projs = []
X_test_projs = []
for i in range(len(seeds)):
    digit_vector, proj, X_train_copy, X_test_copy = HD_classifiers(seeds[i])
    models.append(digit_vector)
    projs.append(proj)
    X_train_projs.append(X_train_copy)
    X_test_projs.append(X_test_copy)

Seed:  30
Encoding:  float
Generating random projection...
(10000, 784)
Projecting images to higher dim space...
(40200, 784)
(10000, 784)
Dimension of digit vector:  (10, 10000)
Train accuracy:  0.8133084577114428
(19800, 784)
(10000, 784)
Test accuracy:  0.8077777777777778
Seed:  40
Encoding:  float
Generating random projection...
(10000, 784)
Projecting images to higher dim space...
(40200, 784)
(10000, 784)
Dimension of digit vector:  (10, 10000)
Train accuracy:  0.8139800995024875
(19800, 784)
(10000, 784)
Test accuracy:  0.8082323232323232
Seed:  50
Encoding:  float
Generating random projection...
(10000, 784)
Projecting images to higher dim space...
(40200, 784)
(10000, 784)
Dimension of digit vector:  (10, 10000)
Train accuracy:  0.8131094527363184
(19800, 784)
(10000, 784)
Test accuracy:  0.8085858585858586


In [73]:
len(X_test_projs)

3

In [74]:
X_perturb_images_projs = np.load("./perturbed_images/X_perturb_images_projs.npy")
y_perturb_true = np.load("./perturbed_images/y_perturb_true.npy")

In [75]:
X_perturb_images_projs[0][0].shape

(1, 10000)

In [76]:
X_perturb_images_projs[0][0][0].shape

(10000,)

In [77]:
X_perturb_images_projs[0].shape

(3, 1, 10000)

In [78]:
temp = []
for i in range(len(seeds)):
    temp_image = [X_perturb_images_projs[j][i][0] for j in range(len(X_perturb_images_projs))]
    temp.append(np.array(temp_image))

In [87]:
x = np.append(X_test_projs[0], temp[0], axis = 0)

In [94]:
y_test = np.array(y_test)
y = np.append(y_test, y_perturb_true, axis=0)

In [97]:
y

array([7, 3, 8, ..., 6, 9, 3], dtype=int64)

In [80]:
temp[0]

array([[69.52297951, 76.65971574, 73.72660634, ..., 72.76863243,
        71.53361583, 65.66588578],
       [40.81172579, 44.85320457, 45.85060386, ..., 44.60800724,
        42.42705694, 39.24103954],
       [39.0836685 , 41.2429528 , 45.99120426, ..., 40.9712192 ,
        41.91994388, 39.80646327],
       ...,
       [72.7705554 , 81.10962217, 77.60049694, ..., 72.72229739,
        77.38436877, 71.12960176],
       [44.17017037, 45.53157198, 44.5973116 , ..., 41.63601494,
        41.08704863, 41.45670503],
       [42.36055199, 46.97993197, 45.30490488, ..., 45.21467243,
        42.33299272, 40.24735592]])

In [50]:
for i in range(len(X_perturb_images_projs)):
    print(X_perturb_images_projs[i].shape)

(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1,

(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1, 10000)
(3, 1,

In [48]:
temp = []
for i in range(len(seeds)):
    temp_image = [X_per[j][i][0] for  in X_perturb_images_projs]

[array([69.52297951, 76.65971574, 73.72660634, ..., 72.76863243,
        71.53361583, 65.66588578]),
 array([40.81172579, 44.85320457, 45.85060386, ..., 44.60800724,
        42.42705694, 39.24103954]),
 array([39.0836685 , 41.2429528 , 45.99120426, ..., 40.9712192 ,
        41.91994388, 39.80646327]),
 array([42.66946253, 46.81475359, 45.31970835, ..., 43.90959579,
        44.99219651, 41.08608981]),
 array([32.939906  , 33.12064926, 35.09852035, ..., 30.73735092,
        34.77790206, 30.56143914]),
 array([40.44897112, 45.97429021, 47.14105935, ..., 44.28936771,
        44.92977144, 38.83923827]),
 array([23.32741316, 23.92003981, 24.99819011, ..., 25.10107593,
        25.07858817, 25.15304743]),
 array([39.32507867, 46.70126472, 43.96667035, ..., 42.98884994,
        41.1579287 , 40.35018873]),
 array([65.42035197, 67.45602734, 64.41938991, ..., 64.92146799,
        64.60479399, 62.51682721]),
 array([28.21382154, 32.47321445, 30.45624455, ..., 25.95830745,
        26.55883909, 24.29

In [29]:
X_perturb_images_projs.reshape((2586, 10000))

ValueError: cannot reshape array of size 77580000 into shape (2586,10000)

In [44]:
raw_models = np.load("./models/raw_models.npy")
retrained_models = np.load("./models/retrained_models.npy")

In [45]:
import copy
seeds = [30, 40, 50]
D = 10000 # dimensions in random space
IMG_LEN = 28
results = []

for seed, model in zip(seeds, raw_models):
    np.random.seed(seed)
    proj = np.random.rand(D, IMG_LEN * IMG_LEN)
#     X_train_copy = copy.deepcopy(X_train)
#     X_train_copy = get_scenes(X_train_copy, proj)
    X_test_copy = copy.deepcopy(X_test)
    X_test_copy = get_scenes(X_test_copy, proj)
    predictions = classify(X_test_copy, model)
    results.append(predictions)
    acc = accuracy_score(y_test[:X_test.shape[0]], predictions)
    print(acc)
    print("here")

(10000, 784)
(10000, 784)
0.8192
here
(10000, 784)
(10000, 784)
0.8184
here
(10000, 784)
(10000, 784)
0.8196
here


In [46]:
import pandas as pd
df = pd.DataFrame({'model_30': list(results[0]),
                   'model_40': list(results[1]),
                   'model_50': list(results[2]),
                   'y': y_test})

In [47]:
df_discrepencies = df[(df["model_30"] != df["model_40"]) | (df["model_30"] != df["model_50"]) | (df["model_40"] != df["model_50"])]

In [48]:
len(df_discrepencies)

311

plt.imshow(X_train[2].reshape(28, 28), interpolation='nearest')

img = Image.fromarray(X_train[2].reshape(28, 28))
img.show()

In [5]:
D = 10000 # dimensions in random space
IMG_LEN = 28
NUM_SAMPLES = X_train.shape[0]

In [35]:
# print("Generating random projection...")
# proj = np.random.rand(D, IMG_LEN * IMG_LEN)
print("Generating random projection...")
seed = 50
np.random.seed(seed)
proj = np.random.rand(D, IMG_LEN * IMG_LEN)
# proj[proj==0] = -1
print(proj.shape)
def get_scene(img, proj):
    return np.dot(proj, img)

# Transform the image vectors into the hypervectors
def get_scenes(images, proj):
    print(images.shape)
    print(proj.shape)
    return np.dot(images[:NUM_SAMPLES, :], proj.T)

print("Projecting images to higher dim space...")
X_train = get_scenes(X_train, proj)

Generating random projection...
(10000, 784)
Projecting images to higher dim space...
(40200, 784)
(10000, 784)


In [11]:
print(digit_vectors.shape)

NameError: name 'digit_vectors' is not defined

In [ ]:
digit_vectors = np.zeros((10, D))
# num_count = {}
for i in range(NUM_SAMPLES):
#     num_count[y_train[i]] =  num_count.get(y_train[i], 0) + 1
    digit_vectors[y_train[i]] += X_train[i]
digit_vectors = np.array(digit_vectors)

In [ ]:
# digit_vectors[digit_vectors > 0] = 1
# digit_vectors[digit_vectors <= 0] = -1

In [ ]:
digit_vectors.shape

for i in num_count:
    digit_vectors[i] /= num_count[i] 

In [36]:
def classify(images, digit_vectors):
    similarities = cosine_similarity(images, digit_vectors)
    classifications = np.argmax(similarities, axis=1)
    return classifications

In [ ]:
print("Train accuracy:")
predictions = classify(X_train, digit_vectors)
acc = accuracy_score(y_train[:X_train.shape[0]], predictions)
print(acc)

In [ ]:
print("Test accuracy:")
X_test = get_scenes(X_test, proj)
predictions = classify(X_test, digit_vectors)
acc = accuracy_score(y_test[:X_test.shape[0]], predictions)
print(acc)

import pandas as pd
df = pd.DataFrame({f'model_{seed}': predictions,
                   'y': y_train})

In [60]:
np.save(f'./temp/digit_vectors_seed_{seed}_version_2_floating.npy', digit_vectors)

#### Make sure you sun from top until this point with multiple seeds to run code below this point

In [37]:
model_30 = np.load("./temp/digit_vectors_seed_30_version_2_floating.npy")
model_40 = np.load("./temp/digit_vectors_seed_40_version_2_floating.npy")
model_50 = np.load("./temp/digit_vectors_seed_50_version_2_floating.npy")

### First, retraining on Adversarial inputs from Training set

In [38]:
X_train, labels_train, _, _ = load_dataset()
# X_train, labels_train = shuffle(X_train, labels_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, labels_train, test_size=0.33, random_state=42)

In [39]:
import copy
seeds = [30, 40, 50]
models = [model_30, model_40, model_50]
results = []

for seed, model in zip(seeds, models):
    np.random.seed(seed)
    proj = np.random.rand(D, IMG_LEN * IMG_LEN)
#     X_train_copy = copy.deepcopy(X_train)
#     X_train_copy = get_scenes(X_train_copy, proj)
    X_test_copy = copy.deepcopy(X_test)
    X_test_copy = get_scenes(X_test_copy, proj)
    predictions = classify(X_test_copy, model)
    results.append(predictions)
    print("here")

(19800, 784)
(10000, 784)
here
(19800, 784)
(10000, 784)
here
(19800, 784)
(10000, 784)
here


In [40]:
len(results[0])

19800

In [41]:
import pandas as pd
df = pd.DataFrame({'model_30': list(results[0]),
                   'model_40': list(results[1]),
                   'model_50': list(results[2]),
                   'y': y_test})

In [42]:
df.head()

,model_30,model_40,model_50,y
0,7,7,7,7
1,3,3,3,3
2,8,8,8,8
3,9,9,9,9
4,3,3,3,3


In [43]:
df_discrepencies = df[(df["model_30"] != df["model_40"]) | (df["model_30"] != df["model_50"]) | (df["model_40"] != df["model_50"])]

In [44]:
df_discrepencies.head()

,model_30,model_40,model_50,y
15,9,7,9,7
29,2,6,2,6
64,5,0,5,0
83,1,1,7,9
102,5,5,8,5


In [45]:
len(df_discrepencies)

631

In [190]:
df_discrepencies.reset_index(inplace=True)

In [191]:
df_discrepencies.head()

,index,model_30,model_40,model_50,y
0,0,7,9,7,8
1,2,4,4,2,9
2,51,9,4,9,4
3,151,0,0,4,7
4,175,9,7,9,7


In [192]:
def get_scene(img, proj):
    return np.dot(img, proj.T)

In [193]:
epochs = 10

for epoch in range(epochs):
    for row in df_discrepencies.iterrows():
        idx = row[1]["index"]
        y_false = row[1]["model_50"]
        y_true = row[1]["y"]
        hv = get_scene(X_train[idx].reshape((1, -1)), proj)
        model_50[y_false] -= hv[0]
        model_50[y_true] += hv[0]

In [71]:
for i, j, k in zip([1, 2, 3], ['a', 'b', 'c'], [4, 5, 6]):
    print(i, j, k)

1 a 4
2 b 5
3 c 6


In [194]:
# print("Generating random projection...")
# proj = np.random.rand(D, IMG_LEN * IMG_LEN)
print("Generating random projection...")
seed = 50
np.random.seed(seed)
proj = np.random.rand(D, IMG_LEN * IMG_LEN)
# proj[proj==0] = -1
print(proj.shape)
def get_scene(img, proj):
    return np.dot(img, proj.T)

# Transform the image vectors into the hypervectors
def get_scenes(images, proj):
    print(images.shape)
    print(proj.shape)
    return np.dot(images[:NUM_SAMPLES, :], proj.T)

print("Projecting images to higher dim space...")
X_train = get_scenes(X_train, proj)

Generating random projection...
(10000, 784)
Projecting images to higher dim space...
(40200, 784)
(10000, 784)


In [195]:
print("Train accuracy:")
predictions = classify(X_train, model_50)
acc = accuracy_score(y_train[:X_train.shape[0]], predictions)
print(acc)

Train accuracy:
0.8420398009950248


In [196]:
print("Test accuracy:")
X_test = get_scenes(X_test, proj)
predictions = classify(X_test, model_50)
acc = accuracy_score(y_test[:X_test.shape[0]], predictions)
print(acc)

Test accuracy:
(19800, 784)
(10000, 784)
0.8433838383838383


In [201]:
test = df[(df["model_30"] == df["model_40"]) & (df["model_30"] == df["model_50"]) & (df["model_40"] == df["model_50"]) ]

In [208]:
1 - len(test[test["model_50"] != test["y"]])/len(df)

0.8320398009950248

In [ ]:
np.save(f'./temp/digit_vectors_seed_{seed}_version_2_floating_retrained.npy', digit_vectors)

In [ ]:
model_30 = np.load("./temp/digit_vectors_seed_30_version_2_floating_retrained.npy")
model_40 = np.load("./temp/digit_vectors_seed_40_version_2_floating_retrained.npy")
model_50 = np.load("./temp/digit_vectors_seed_50_version_2_floating_retrained.npy")

In [ ]:
X_train, labels_train, _, _ = load_dataset()
# X_train, labels_train = shuffle(X_train, labels_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, labels_train, test_size=0.33, random_state=42)

In [ ]:
import copy
seeds = [30, 40, 50]
models = [model_30, model_40, model_50]
results = []

for seed, model in zip(seeds, models):
    np.random.seed(seed)
    proj = np.random.rand(D, IMG_LEN * IMG_LEN)
    X_train_copy = copy.deepcopy(X_train)
    X_train_copy = get_scenes(X_train_copy, proj)
    predictions = classify(X_train_copy, model)
    results.append(predictions)
    print("here")

In [49]:
X_train, labels_train, _, _ = load_dataset()
# X_train, labels_train = shuffle(X_train, labels_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, labels_train, test_size=0.33, random_state=42)

In [23]:
import copy
seeds = [30, 40, 50]
models = [model_30, model_40, model_50]
results = []

for seed, model in zip(seeds, models):
    np.random.seed(seed)
    proj = np.random.rand(D, IMG_LEN * IMG_LEN)
    X_test_copy = copy.deepcopy(X_test)
    X_test_copy = get_scenes(X_test_copy, proj)
    predictions = classify(X_test_copy, model)
    results.append(predictions)
    print("here")

NameError: name 'get_scenes' is not defined

In [49]:
np.save("./temp/results_30_40_50_version_2.npy", results)

In [4]:
results = np.load("./temp/results_30_40_50_version_2.npy")

In [5]:
import pandas as pd
df = pd.DataFrame({'model_30': list(results[0]),
                   'model_40': list(results[1]),
                   'model_50': list(results[2]),
                   'y': y_test})

In [6]:
df.head()

,model_30,model_40,model_50,y
0,7,7,7,7
1,3,3,3,3
2,8,8,8,8
3,9,9,9,9
4,3,3,3,3


In [7]:
len(df)

19800

In [8]:
df_discrepencies = df[(df["model_30"] != df["model_40"]) | (df["model_30"] != df["model_50"]) | (df["model_40"] != df["model_50"])]

In [9]:
df_discrepencies.to_excel("./temp/test_discrepencies_version_2_floating.xlsx")

df_discrepencies = df[(df["y"] == df["model_30"]) | (df["y"] == df["model_40"]) | (df["y"] == df["model_50"])]

df_discrepencies = df_discrepencies[((df_discrepencies["y"] + df_discrepencies["model_30"]
                                     + df_discrepencies["model_40"] + df_discrepencies["model_50"]) 
                                     != df_discrepencies["y"] * 4)]

df_discrepencies.head()

sum(df_discrepencies["model_50"] != df_discrepencies["y"])

len(df_discrepencies)

df_discrepencies.to_excel("./temp/test_discrepencies_version_2.xlsx")

test_discrepencies = pd.read_excel("./temp/test_discrepencies_version_2.xlsx")

In [10]:
len(df_discrepencies)

631

In [11]:
df_discrepencies.head()

,model_30,model_40,model_50,y
15,9,7,9,7
29,2,6,2,6
64,5,0,5,0
83,1,1,7,9
102,5,5,8,5


In [60]:
test_discrepencies.rename(columns={"Unnamed: 0": "idx"}, inplace=True)

In [61]:
test_discrepencies.head()

,idx,model_30,model_40,model_50,y
0,15,9,7,9,7
1,29,2,6,2,6
2,64,5,0,5,0
3,102,5,5,8,5
4,173,9,7,9,9


In [62]:
def get_scene(img, proj):
    return np.dot(img, proj.T)

In [63]:
for row in test_discrepencies.iterrows():
    idx = row[1]["idx"]
    y_false = row[1]["model_30"]
    y_true = row[1]["y"]
    hv = get_scene(X_train[idx].reshape((1, -1)), proj)
    model_30[y_false] -= hv[0]
    model_30[y_true] += hv[0]

In [64]:
# print("Generating random projection...")
# proj = np.random.rand(D, IMG_LEN * IMG_LEN)
print("Generating random projection...")
seed = 30
np.random.seed(seed)
proj = np.random.rand(D, IMG_LEN * IMG_LEN)
# proj[proj==0] = -1
print(proj.shape)
def get_scene(img, proj):
    return np.dot(img, proj.T)

# Transform the image vectors into the hypervectors
def get_scenes(images, proj):
    print(images.shape)
    print(proj.shape)
    return np.dot(images[:NUM_SAMPLES, :], proj.T)

print("Projecting images to higher dim space...")
X_train = get_scenes(X_train, proj)

Generating random projection...
(10000, 784)
Projecting images to higher dim space...
(40200, 784)
(10000, 784)


In [65]:
print("Train accuracy:")
predictions = classify(X_train, model_30)
acc = accuracy_score(y_train[:X_train.shape[0]], predictions)
print(acc)

Train accuracy:
0.8130597014925374


In [66]:
print("Test accuracy:")
X_test = get_scenes(X_test, proj)
predictions = classify(X_test, model_30)
acc = accuracy_score(y_test[:X_test.shape[0]], predictions)
print(acc)

Test accuracy:
(19800, 784)
(10000, 784)
0.8077272727272727
